In [ ]:
def area_segmentation(x_dir, y_dir, classes=['road','pavement'],
                     BACKBONE = 'efficientnetb3', BATCH_SIZE = 4, LR = 0.0001, EPOCHS = 40):
    
    import cv2
    import keras
    import numpy as np
    import matplotlib.pyplot as plt
    from PIL import Image
    from sklearn.model_selection import train_test_split
    
    import segmentation_models as sm
    from preprocessing import *
    from augmentation import *
    
    def ds_generate(x_dir, y_dir, BACKBONE = BACKBONE,Classes=classes):
        ids = os.listdir(x_dir)
        X = [os.path.join(x_dir, image_id) for image_id in ids]
        y = [os.path.join(y_dir, image_id) for image_id in ids]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
        
        preprocess_input = sm.get_preprocessing(BACKBONE)
        
        train_dataset = Dataset(
            X_train, 
            y_train, 
            classes=CLASSES, 
            augmentation=get_training_augmentation(),
            preprocessing=get_preprocessing(preprocess_input))

        valid_dataset = Dataset(
            X_val, 
            y_val, 
            classes=CLASSES, 
            augmentation=get_validation_augmentation(),
            preprocessing=get_preprocessing(preprocess_input))
        
        test_dataset = test_Dataset(
            X_test, 
            y_test, 
            classes=CLASSES, 
            augmentation=get_validation_augmentation(),
            preprocessing=get_preprocessing(preprocess_input))
        
        train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        valid_dataloader = Dataloder(valid_dataset, batch_size=1, shuffle=False)
        test_dataloader = Dataloder(test_dataset, batch_size=1, shuffle=False)
        
        assert train_dataloader[0][0].shape == (BATCH_SIZE, 320, 320, 3)
        assert train_dataloader[0][1].shape == (BATCH_SIZE, 320, 320, n_classes)
        
        return train_dataloader, valid_dataloader, test_dataloader
        
        
    def train(train_dataloader, valid_dataloader, test_dataloader, BACKBONE = BACKBONE,
              Classes=classes, BATCH_SIZE = BATCH_SIZE, LR = LR, EPOCHS = EPOCHS):

        # define network parameters
        n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
        activation = 'sigmoid' if n_classes == 1 else 'softmax'

        #create model
        model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)
        
        # define optomizer
        optim = keras.optimizers.Adam(LR)

        # Segmentation models losses can be combined together by '+' and scaled by integer or float factor
        dice_loss = sm.losses.DiceLoss(class_weights=np.array([1, 2, 0.5]))
        focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
        total_loss = dice_loss + (1 * focal_loss)

        # metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
        metrics = ['accuracy']

        # compile keras model with defined optimozer, loss and metrics
        model.compile(optim, total_loss, metrics)
        
        callbacks = [
            keras.callbacks.ModelCheckpoint('./best_model_segmentation.h5', save_weights_only=True, save_best_only=True, mode='min'),
            keras.callbacks.ReduceLROnPlateau(),
            keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
        
        history = model.fit_generator(
            train_dataloader,
            steps_per_epoch=len(train_dataloader),
            epochs=EPOCHS,
            callbacks=callbacks,
            validation_data=valid_dataloader,
            validation_steps=len(valid_dataloader))
        
        model.load_weights('best_model_segmentation.h5') 
        
        scores = model.evaluate_generator(test_dataloader)

        print("Loss: {:.5}".format(scores[0]))
        for metric, value in zip(metrics, scores[1:]):
            print("mean {}: {:.5}".format(metric, value))
            
    train_dataloader, valid_dataloader, test_dataloader = ds_generate(x_dir, y_dir, BACKBONE = BACKBONE,Classes=classes)
    train(train_dataloader, valid_dataloader, test_dataloader, BACKBONE = BACKBONE,
          Classes=classes, BATCH_SIZE = BATCH_SIZE, LR = LR, EPOCHS = EPOCHS)